# HW3: Netflix Data Analysis

In this homework assignment, you will analyze the netflix prize data. The data consist of 100,480,50 movie ratings on a scale from 0 to 5 stars. The reveiws are distributed across 17,770 movies and 480,189. We have provided the training data as a sparse matrix where the row corresponds to the movie ID and the column corresponds to the user ID. A seperate file contains the title and year of release for each movie. The original, raw data consists of multiple lists of tuples; each list is a separate movie and each tuple is User ID, Rating, and Rating Year. 
The original data can be downloaded here: https://archive.org/download/nf_prize_dataset.tar
Further information about the netflix prize is available online: 
https://en.wikipedia.org/wiki/Netflix_Prize
https://www.netflixprize.com/

In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy.sparse
from scipy import stats
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN, SpectralClustering, AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.model_selection import train_test_split


In [3]:
# This file consists of titles and release years associated with each ID
movie_titles = pd.read_csv('movie_titles.txt', header = None, names = ['ID','Year','Name'], encoding='latin-1')
print(movie_titles.head())
print(movie_titles.shape)

movie_by_id = {}
for id, name, year in zip(movie_titles['ID'], movie_titles['Name'], movie_titles['Year']):
    if not (np.isnan(year)):
        year = str(int(year))
    else:
        year = 'NaN'
    movie_by_id[id] = name + ' ' + '(' + year + ')'

   ID    Year                          Name
0   1  2003.0               Dinosaur Planet
1   2  2004.0    Isle of Man TT 2004 Review
2   3  1997.0                     Character
3   4  1994.0  Paula Abdul's Get Up & Dance
4   5  2004.0      The Rise and Fall of ECW
(17770, 3)


In [4]:
# This file is a sparse matrix of movies by user, with each element a rating (1-5) or nonresponse (0)
ratings_csr = scipy.sparse.load_npz('netflix_full_csr.npz')
print(ratings_csr.shape)

(17771, 2649430)


To avoid memory overflow errors we have randomly subsampled the data. Some computers can handle the full dataset (e.g. 2017 Macbook Pro can perform SVD on the full dataset). Older computers likely need to subsample the data. You can consider using Princeton computing resources and clusters to perform more computationally expensive analysis.

In [5]:
#n_samples = 5000
#n_viewers = 500000
#random_sample_movies = np.random.choice(17771, size = n_samples)
#random_sample_viewers = np.random.choice(2649430, size = n_viewers)
#ratings_small = ratings_csr[:,random_sample_viewers]

In [80]:
# Filter the matrix to remove rows with NO REVIEWS
start = time.time()
ratings_csc = ratings_csr.T
print('before removing users with no reviews: ', ratings_csc.shape)
non_zero_users_csc = ratings_csc[(ratings_csc.getnnz(axis=1) != 0)]
print(non_zero_users_csc.shape)

finish = time.time()
print('finished reduction in %.2f seconds' % (finish - start))
ratings_small = non_zero_users_csc

before removing users with no reviews:  (2649430, 17771)
(480189, 17771)
finished reduction in 23.21 seconds
(480189, 10)
finished svd in 19.60 seconds


In [144]:
start = time.time()
svd = TruncatedSVD(n_components = 15, algorithm="arpack", random_state=0)
all_users_small = svd.fit_transform(ratings_small)
finish = time.time()
print(all_users_small.shape)
print('finished svd in %.2f seconds' % (finish - start))

(480189, 15)
finished svd in 29.56 seconds


In [145]:
start = time.time()
kmeans_all_users = KMeans(n_clusters = 20 , random_state=0, algorithm="full")
kmeans_all_users.fit(all_users_small)
finish = time.time()
print('finished clustering in %.2f seconds' % (finish - start))
clusters_all_users = kmeans_all_users.labels_
clusters, counts = np.unique(clusters_all_users, return_counts=True)
print(counts)

finished clustering in 254.06 seconds
[  6760 160940   6160   5905  16404   2944  21565   7315   4781  18507
  11498  44379  13283   7477  13423  27054  62848   2864  34190  11892]


A common methods for analyzing large datasets is dimension reduction. Here we perform a truncated SVD suited for sparse datasets and analyze which movies are associated with different latent dimensions

In [149]:
y_pred_mode = []
y_pred_mean = []
y_test_all = []
for id in range(1777):
    movie_id = id
    num_reviews = ratings_small[:,movie_id].count_nonzero()
    movie_name = movie_titles[movie_titles['ID'] == movie_id]['Name']
    print ('Movie #%s, %s, average rating: %.2f in %i reviews' % (
        movie_id,
        movie_name,
        np.sum(ratings_small[:,movie_id]) / num_reviews,
        int(num_reviews)
    ))
    
    start = time.time()
    filter_by = np.ravel((ratings_small[:,movie_id] != 0.0).toarray())
    filtered_clusters = clusters_all_users[filter_by]
    filtered_ratings = ratings_small[filter_by,:]
    
    movie_mask = np.ravel(np.full((filtered_ratings.shape[1], 1), True))
    movie_mask[movie_id] = False
    X = filtered_ratings[:,movie_mask]
    y = filtered_ratings[:,movie_id].toarray()

    
    #y_and_c = np.transpose(np.vstack([np.ravel(y), filtered_clusters]))
    print(X.shape)
    print(y.shape)
    #print(y_and_c.shape)
    
    X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(X, y, filtered_clusters, test_size=0.2, random_state=0)
    
    for (cluster, review) in zip(c_test, y_test):
        y_test_all.append(review)
        cluster_reviews = y_train[c_train == cluster]
        mode, _ = stats.mode(y_train, axis =None)
        mode_all = mode[0]
        mean_all = np.mean(y_train)
        
        if len(cluster_reviews) != 0:
            mode, _ = stats.mode(cluster_reviews, axis =None)
            y_pred_mode.append(mode[0])
            y_pred_mean.append(np.mean(cluster_reviews))
        else:
            mode, _ = stats.mode(y_train, axis =None)
            y_pred_mode.append(mode_all)
            y_pred_mean.append(mean_all)
    
print('Variance score mean: %.2f' % r2_score(y_test_all, y_pred_mean))
print('Variance score mode: %.2f' % r2_score(y_test_all, y_pred_mode))
print('MSE mean: %.2f' % mean_squared_error(y_test_all, y_pred_mean))
print('MSE mode: %.2f' % mean_squared_error(y_test_all, y_pred_mode))
print()
    
        
    


/Users/Cat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Movie #0, Series([], Name: Name, dtype: object), average rating: nan in 0 reviews
(0, 17770)
(0, 1)
Movie #1, 0    Dinosaur Planet
Name: Name, dtype: object, average rating: 3.75 in 547 reviews
(547, 17770)
(547, 1)
Movie #2, 1    Isle of Man TT 2004 Review
Name: Name, dtype: object, average rating: 3.56 in 145 reviews
(145, 17770)
(145, 1)
Movie #3, 2    Character
Name: Name, dtype: object, average rating: 3.64 in 2012 reviews
(2012, 17770)
(2012, 1)
Movie #4, 3    Paula Abdul's Get Up & Dance
Name: Name, dtype: object, average rating: 2.74 in 142 reviews
(142, 17770)
(142, 1)
Movie #5, 4    The Rise and Fall of ECW
Name: Name, dtype: object, average rating: 3.92 in 1140 reviews
(1140, 17770)
(1140, 1)
Movie #6, 5    Sick
Name: Name, dtype: object, average rating: 3.08 in 1019 reviews
(1019, 17770)
(1019, 1)
Movie #7, 6    8 Man
Name: Name, dtype: object, average rating: 2.13 in 93 reviews
(93, 17770)
(93, 1)
Movie #8, 7    What the #$*! Do We Know!?
Name: Name, dtype: object, average

(258, 17770)
(258, 1)
Movie #66, 65    Barbarian Queen 2
Name: Name, dtype: object, average rating: 2.30 in 166 reviews
(166, 17770)
(166, 1)
Movie #67, 66    Vampire Journals
Name: Name, dtype: object, average rating: 2.56 in 289 reviews
(289, 17770)
(289, 1)
Movie #68, 67    Invader Zim
Name: Name, dtype: object, average rating: 4.14 in 2216 reviews
(2216, 17770)
(2216, 1)
Movie #69, 68    WWE: Armageddon 2003
Name: Name, dtype: object, average rating: 3.29 in 116 reviews
(116, 17770)
(116, 1)
Movie #70, 69    Tai Chi: The 24 Forms
Name: Name, dtype: object, average rating: 2.73 in 343 reviews
(343, 17770)
(343, 1)
Movie #71, 70    Maya Lin: A Strong Clear Vision
Name: Name, dtype: object, average rating: 3.77 in 1561 reviews
(1561, 17770)
(1561, 1)
Movie #72, 71    At Home Among Strangers
Name: Name, dtype: object, average rating: 3.15 in 179 reviews
(179, 17770)
(179, 1)
Movie #73, 72    Davy Crockett: 50th Anniversary Double Feature
Name: Name, dtype: object, average rating: 3.89 

(830, 17770)
(830, 1)
Movie #133, 132    Viva La Bam: Season 1
Name: Name, dtype: object, average rating: 3.74 in 2563 reviews
(2563, 17770)
(2563, 1)
Movie #134, 133    Spirit Lost
Name: Name, dtype: object, average rating: 2.13 in 99 reviews
(99, 17770)
(99, 1)
Movie #135, 134    GTO: Great Teacher Onizuka: Set 2
Name: Name, dtype: object, average rating: 4.14 in 384 reviews
(384, 17770)
(384, 1)
Movie #136, 135    Cat and the Canary
Name: Name, dtype: object, average rating: 2.98 in 120 reviews
(120, 17770)
(120, 1)
Movie #137, 136    Naked Lies
Name: Name, dtype: object, average rating: 2.39 in 204 reviews
(204, 17770)
(204, 1)
Movie #138, 137    Star Trek: Voyager: Season 1
Name: Name, dtype: object, average rating: 3.94 in 6007 reviews
(6007, 17770)
(6007, 1)
Movie #139, 138    Allergies: A Natural Approach
Name: Name, dtype: object, average rating: 2.51 in 96 reviews
(96, 17770)
(96, 1)
Movie #140, 139    Lost in the Wild
Name: Name, dtype: object, average rating: 2.53 in 119 re

(81260, 17770)
(81260, 1)
Movie #198, 197    Gupt
Name: Name, dtype: object, average rating: 3.08 in 194 reviews
(194, 17770)
(194, 1)
Movie #199, 198    The Deer Hunter
Name: Name, dtype: object, average rating: 3.89 in 35509 reviews
(35509, 17770)
(35509, 1)
Movie #200, 199    The Fall of the Roman Empire
Name: Name, dtype: object, average rating: 3.01 in 129 reviews
(129, 17770)
(129, 1)
Movie #201, 200    Home Movie
Name: Name, dtype: object, average rating: 3.42 in 2724 reviews
(2724, 17770)
(2724, 1)
Movie #202, 201    Ruby's Bucket of Blood
Name: Name, dtype: object, average rating: 3.10 in 382 reviews
(382, 17770)
(382, 1)
Movie #203, 202    Sports Illustrated Swimsuit Edition: 1997
Name: Name, dtype: object, average rating: 3.16 in 96 reviews
(96, 17770)
(96, 1)
Movie #204, 203    Venus Boyz
Name: Name, dtype: object, average rating: 2.71 in 230 reviews
(230, 17770)
(230, 1)
Movie #205, 204    Troy: Bonus Material
Name: Name, dtype: object, average rating: 3.30 in 513 reviews


...

(1460, 17770)
(1460, 1)
Movie #1489, 1488    American Adobo
Name: Name, dtype: object, average rating: 2.83 in 479 reviews
(479, 17770)
(479, 1)
Movie #1490, 1489    Knight Hunters Eternity
Name: Name, dtype: object, average rating: 3.38 in 225 reviews
(225, 17770)
(225, 1)
Movie #1491, 1490    Dream Theater: Live at Budokan
Name: Name, dtype: object, average rating: 3.84 in 302 reviews
(302, 17770)
(302, 1)
Movie #1492, 1491    Rhinestone
Name: Name, dtype: object, average rating: 2.21 in 755 reviews
(755, 17770)
(755, 1)
Movie #1493, 1492    John Lee Hooker: Live in Montreal: Montreal Ja...
Name: Name, dtype: object, average rating: 3.12 in 135 reviews
(135, 17770)
(135, 1)
Movie #1494, 1493    Appalachian Journey: Yo-Yo Ma/Edgar Meyer/Mark...
Name: Name, dtype: object, average rating: 3.47 in 240 reviews
(240, 17770)
(240, 1)
Movie #1495, 1494    Alias: Season 1
Name: Name, dtype: object, average rating: 4.34 in 16683 reviews
(16683, 17770)
(16683, 1)
Movie #1496, 1495    Korn: Korn

(6258, 17770)
(6258, 1)
Movie #1554, 1553    The 5th Musketeer
Name: Name, dtype: object, average rating: 2.70 in 103 reviews
(103, 17770)
(103, 1)
Movie #1555, 1554    The Little Fugitive
Name: Name, dtype: object, average rating: 3.16 in 185 reviews
(185, 17770)
(185, 1)
Movie #1556, 1555    A Farewell to Arms
Name: Name, dtype: object, average rating: 3.20 in 1045 reviews
(1045, 17770)
(1045, 1)
Movie #1557, 1556    Dolls
Name: Name, dtype: object, average rating: 3.05 in 428 reviews
(428, 17770)
(428, 1)
Movie #1558, 1557    Rocky V
Name: Name, dtype: object, average rating: 3.03 in 8170 reviews
(8170, 17770)
(8170, 1)
Movie #1559, 1558    AFI's 100 Years
Name: Name, dtype: object, average rating: 3.27 in 361 reviews
(361, 17770)
(361, 1)
Movie #1560, 1559    Disney Princess Stories: Vol. 1: A Gift From t...
Name: Name, dtype: object, average rating: 3.33 in 643 reviews
(643, 17770)
(643, 1)
Movie #1561, 1560    American Wedding
Name: Name, dtype: object, average rating: 3.48 in 61

(652, 17770)
(652, 1)
Movie #1618, 1617    Nausicaa of the Valley of the Wind
Name: Name, dtype: object, average rating: 4.19 in 6674 reviews
(6674, 17770)
(6674, 1)
Movie #1619, 1618    The Journey of Natty Gann
Name: Name, dtype: object, average rating: 3.76 in 1044 reviews
(1044, 17770)
(1044, 1)
Movie #1620, 1619    The Saddle Club: The First Adventure
Name: Name, dtype: object, average rating: 3.41 in 318 reviews
(318, 17770)
(318, 1)
Movie #1621, 1620    Richard Kern: Hardcore Collection
Name: Name, dtype: object, average rating: 2.34 in 181 reviews
(181, 17770)
(181, 1)
Movie #1622, 1621    Zhou Yu's Train
Name: Name, dtype: object, average rating: 2.89 in 875 reviews
(875, 17770)
(875, 1)
Movie #1623, 1622    Pokemon Advanced
Name: Name, dtype: object, average rating: 3.45 in 253 reviews
(253, 17770)
(253, 1)
Movie #1624, 1623    The Minion
Name: Name, dtype: object, average rating: 2.46 in 127 reviews
(127, 17770)
(127, 1)
Movie #1625, 1624    Aliens: Collector's Edition
Name:

(147, 17770)
(147, 1)
Movie #1681, 1680    The Tesseract
Name: Name, dtype: object, average rating: 2.54 in 214 reviews
(214, 17770)
(214, 1)
Movie #1682, 1681    Absolute Power
Name: Name, dtype: object, average rating: 3.69 in 25961 reviews
(25961, 17770)
(25961, 1)
Movie #1683, 1682    The Lion King: Circle of Life: Sing-Along Songs
Name: Name, dtype: object, average rating: 3.01 in 686 reviews
(686, 17770)
(686, 1)
Movie #1684, 1683    What Alice Found
Name: Name, dtype: object, average rating: 2.76 in 571 reviews
(571, 17770)
(571, 1)
Movie #1685, 1684    First Option
Name: Name, dtype: object, average rating: 2.50 in 74 reviews
(74, 17770)
(74, 1)
Movie #1686, 1685    Riding the Rails: American Experience
Name: Name, dtype: object, average rating: 3.62 in 551 reviews
(551, 17770)
(551, 1)
Movie #1687, 1686    Eloise at the Plaza
Name: Name, dtype: object, average rating: 3.46 in 1579 reviews
(1579, 17770)
(1579, 1)
Movie #1688, 1687    To Sleep With a Vampire
Name: Name, dtype: o

(8588, 17770)
(8588, 1)
Movie #1744, 1743    Beverly Hills Cop
Name: Name, dtype: object, average rating: 3.82 in 73004 reviews
(73004, 17770)
(73004, 1)
Movie #1745, 1744    Love Me
Name: Name, dtype: object, average rating: 3.01 in 309 reviews
(309, 17770)
(309, 1)
Movie #1746, 1745    Bob Roberts
Name: Name, dtype: object, average rating: 3.41 in 11039 reviews
(11039, 17770)
(11039, 1)
Movie #1747, 1746    Hello Down There
Name: Name, dtype: object, average rating: 3.27 in 129 reviews
(129, 17770)
(129, 1)
Movie #1748, 1747    Happily Ever After: Fairy Tales for Every Child
Name: Name, dtype: object, average rating: 3.15 in 407 reviews
(407, 17770)
(407, 1)
Movie #1749, 1748    Angel and the Badman
Name: Name, dtype: object, average rating: 3.39 in 1319 reviews
(1319, 17770)
(1319, 1)
Movie #1750, 1749    Beast From Haunted Cave/The Brain That Wouldn'...
Name: Name, dtype: object, average rating: 2.40 in 114 reviews
(114, 17770)
(114, 1)
Movie #1751, 1750    L'Ennui
Name: Name, dtyp

In [141]:
movie_results_dict = {}
for id in range(10, 15):
    movie_id = id
    num_reviews = ratings_small[:,movie_id].count_nonzero()
    movie_name = movie_titles[movie_titles['ID'] == movie_id]['Name']
    print ('Movie #%s, %s, average rating: %.2f in %i reviews' % (
        movie_id,
        movie_name,
        np.sum(ratings_small[:,movie_id]) / num_reviews,
        int(num_reviews)
    ))

    start = time.time()
    filter_by = np.ravel((ratings_small[:,movie_id] != 0.0).toarray())
    filtered_ratings = ratings_small[filter_by,:]

    movie_mask = np.ravel(np.full((filtered_ratings.shape[1], 1), True))
    movie_mask[movie_id] = False
    X = filtered_ratings[:,movie_mask]
    y = filtered_ratings[:,movie_id].toarray()
    svd = TruncatedSVD(n_components = np.max([np.min(X.shape)-1, 1]), algorithm="arpack", random_state=0)
    Z = svd.fit_transform(X)
    #Z[Z == 0] = 3.6
    
    
    X_train, X_test, y_train, y_test = train_test_split(Z, y, test_size=0.2, random_state=0)
    print(Z.shape)
    print(y.shape)
    
    
    
    finish = time.time()
    data_time = (finish - start)
    #print 'finished choosing data in %.2f seconds' % data
    for i, name in zip([X_train, X_test, y_train, y_test], ['X_train', 'X_test', 'y_train', 'y_test']):
        print (name, ':', i.shape, '\t',)
    print()
    
    # Run the regression on the film
    start = time.time()
    regr = LinearRegression()
    regr.fit(X_train, y_train)

    y_pred_reg = regr.predict(X_test)

    #print 'Coefficients: \n', regr.coef_
    #print "Mean squared error: %.2f" % mean_squared_error(y_test, y_pred)
    print('No Clustering')
    print('Variance score: %.2f' % r2_score(y_test, y_pred_reg))
    print()
    finish = time.time()
    regr_time = finish - start
   # print ('finished regression in %.2f seconds' % regr_time)
   # print()
    movie_results_dict[id] = {
        'name': str(movie_titles[movie_titles['ID'] == movie_id]['Name']),
        'regr_time': regr_time,
        'data_time': data_time,
        'regr': regr,
        'mse': mean_squared_error(y_test, y_pred_reg),
        'r2': r2_score(y_test, y_pred_reg)
    }
    
    kmeans = KMeans(n_clusters = np.max([int(X_train.shape[0]/50)+1, 3]) , random_state=0, algorithm="full")
    kmeans.fit(X_train)
    cluster_members = kmeans.labels_
    pred_clusters = kmeans.predict(X_test)
    
    y_pred_avg_km = []
    y_pred_rand_km = []
    y_pred_clus_km = []
    y_pred_mode_km = []
    for k in range(len(pred_clusters)):
        person = pred_clusters[k]
        lookalike_y = y_train[np.where(cluster_members==person)]
        y_pred_avg_km.append([np.mean(lookalike_y)])
        y_pred_rand_km.append([np.random.choice(lookalike_y.flatten())])
        y_pred_mode_km.append(stats.mode(lookalike_y)[0][0])
        
        lookalike_x = X_train[np.where(cluster_members==person)]
        regr = LinearRegression()
        regr.fit(lookalike_x, lookalike_y)
        y_pred_clus_km.append(regr.predict(X_test[k].reshape(1, -1))[0])
        
    
    y_pred_avg_km = np.asmatrix(y_pred_avg_km)
    y_pred_mode_km = np.asmatrix(y_pred_mode_km)
    y_pred_rand_km = np.asmatrix(y_pred_rand_km)
    y_pred_clus_km = np.asmatrix(y_pred_clus_km)
    
    print('KMeans Clustering')
    print('Variance score mean: %.2f' % r2_score(y_test, y_pred_avg_km))
    print('Variance score mode: %.2f' % r2_score(y_test, y_pred_mode_km))
    print('Variance score random: %.2f' % r2_score(y_test, y_pred_rand_km))
    print('Variance score regress: %.2f' % r2_score(y_test, y_pred_clus_km))
    print()
    
    dbscan = DBSCAN()
    dbscan.fit(X_train)
    cluster_members = dbscan.labels_
    pred_clusters = dbscan.fit_predict(X_test)
    
    y_pred_avg_db = []
    y_pred_mode_db = []
    y_pred_rand_db = []
    y_pred_clus_db = []
    
    for k in range(len(pred_clusters)):
        person = pred_clusters[k]
        lookalike_y = y_train[np.where(cluster_members==person)]
        y_pred_avg_db.append([np.mean(lookalike_y)])
        y_pred_mode_db.append(stats.mode(lookalike_y)[0][0])
        y_pred_rand_db.append([np.random.choice(lookalike_y.flatten())])
        
        lookalike_x = X_train[np.where(cluster_members==person)]
        regr = LinearRegression()
        regr.fit(lookalike_x, lookalike_y)
        y_pred_clus_db.append(regr.predict(X_test[k].reshape(1, -1))[0])
        
    
    y_pred_avg_db = np.asmatrix(y_pred_avg_db)
    y_pred_mode_db = np.asmatrix(y_pred_mode_db)
    y_pred_rand_db = np.asmatrix(y_pred_rand_db)
    y_pred_clus_db = np.asmatrix(y_pred_clus_db)
    
    print('DBSCAN Clustering')
    print('Variance score mean: %.2f' % r2_score(y_test, y_pred_avg_db))
    print('Variance score mode: %.2f' % r2_score(y_test, y_pred_mode_db))
    print('Variance score random: %.2f' % r2_score(y_test, y_pred_rand_db))
    print('Variance score regress: %.2f' % r2_score(y_test, y_pred_clus_db))
    print()
    
    
    spect = SpectralClustering(n_clusters = np.max([int(X_train.shape[0]/50)+1, 3]) , random_state=0)
    spect.fit(X_train)
    cluster_members = spect.labels_
    pred_clusters = spect.fit_predict(X_test)
    
    y_pred_avg_sc = []
    y_pred_mode_sc = []
    y_pred_rand_sc = []
    y_pred_clus_sc = []
    
    for k in range(len(pred_clusters)):
        person = pred_clusters[k]
        lookalike_y = y_train[np.where(cluster_members==person)]
        y_pred_avg_sc.append([np.mean(lookalike_y)])
        y_pred_mode_sc.append(stats.mode(lookalike_y)[0][0])
        y_pred_rand_sc.append([np.random.choice(lookalike_y.flatten())])
        
        lookalike_x = X_train[np.where(cluster_members==person)]
        regr = LinearRegression()
        regr.fit(lookalike_x, lookalike_y)
        y_pred_clus_sc.append(regr.predict(X_test[k].reshape(1, -1))[0])
        
    
    y_pred_avg_sc = np.asmatrix(y_pred_avg_sc)
    y_pred_mode_sc = np.asmatrix(y_pred_mode_sc)
    y_pred_rand_sc = np.asmatrix(y_pred_rand_sc)
    y_pred_clus_sc = np.asmatrix(y_pred_clus_sc)
    
    print('Spectral Clustering')
    print('Variance score mean: %.2f' % r2_score(y_test, y_pred_avg_sc))
    print('Variance score mode: %.2f' % r2_score(y_test, y_pred_mode_sc))
    print('Variance score random: %.2f' % r2_score(y_test, y_pred_rand_sc))
    print('Variance score regress: %.2f' % r2_score(y_test, y_pred_clus_sc))
    print()
    
    agg = AgglomerativeClustering(n_clusters = np.max([int(X_train.shape[0]/50)+1, 3]))
    agg.fit(X_train)
    cluster_members = agg.labels_
    pred_clusters = agg.fit_predict(X_test)
    
    y_pred_avg_ac = []
    y_pred_mode_ac = []
    y_pred_rand_ac = []
    y_pred_clus_ac = []
    
    for k in range(len(pred_clusters)):
        person = pred_clusters[k]
        lookalike_y = y_train[np.where(cluster_members==person)]
        y_pred_avg_ac.append([np.mean(lookalike_y)])
        y_pred_mode_ac.append(stats.mode(lookalike_y)[0][0])
        y_pred_rand_ac.append([np.random.choice(lookalike_y.flatten())])
        
        lookalike_x = X_train[np.where(cluster_members==person)]
        regr = LinearRegression()
        regr.fit(lookalike_x, lookalike_y)
        y_pred_clus_ac.append(regr.predict(X_test[k].reshape(1, -1))[0])
        
    
    y_pred_avg_ac = np.asmatrix(y_pred_avg_ac)
    y_pred_mode_ac = np.asmatrix(y_pred_mode_ac)
    y_pred_rand_ac = np.asmatrix(y_pred_rand_ac)
    y_pred_clus_ac = np.asmatrix(y_pred_clus_ac)
    
    print('Agglomerative Clustering')
    print('Variance score mean: %.2f' % r2_score(y_test, y_pred_avg_ac))
    print('Variance score mode: %.2f' % r2_score(y_test, y_pred_mode_ac))
    print('Variance score random: %.2f' % r2_score(y_test, y_pred_rand_ac))
    print('Variance score regress: %.2f' % r2_score(y_test, y_pred_clus_ac))
    print()
    
    
    
        
    
    
    
    

Movie #10, 9    Fighter
Name: Name, dtype: object, average rating: 3.18 in 249 reviews
(249, 248)
(249, 1)
X_train : (199, 248) 	
X_test : (50, 248) 	
y_train : (199, 1) 	
y_test : (50, 1) 	

No Clustering
Variance score: 0.07

KMeans Clustering
Variance score mean: 0.01
Variance score mode: -0.01
Variance score random: -0.71
Variance score regress: 0.15

DBSCAN Clustering
Variance score mean: -0.01
Variance score mode: -0.05
Variance score random: -0.68
Variance score regress: 0.07

Spectral Clustering
Variance score mean: -0.08
Variance score mode: -0.26
Variance score random: -1.80
Variance score regress: 0.11

Agglomerative Clustering
Variance score mean: -0.86
Variance score mode: -1.94
Variance score random: -1.15
Variance score regress: -1.57

Movie #11, 10    Full Frame: Documentary Shorts
Name: Name, dtype: object, average rating: 3.03 in 198 reviews


/Users/Cat/anaconda3/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


(198, 197)
(198, 1)
X_train : (158, 197) 	
X_test : (40, 197) 	
y_train : (158, 1) 	
y_test : (40, 1) 	

No Clustering
Variance score: -0.34

KMeans Clustering
Variance score mean: -0.23
Variance score mode: -0.21
Variance score random: -0.82
Variance score regress: -0.44

DBSCAN Clustering
Variance score mean: -0.18
Variance score mode: -0.13
Variance score random: -0.71
Variance score regress: -0.34

Spectral Clustering
Variance score mean: -0.19
Variance score mode: -0.21
Variance score random: -1.45
Variance score regress: -0.32



/Users/Cat/anaconda3/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Agglomerative Clustering
Variance score mean: -0.26
Variance score mode: -0.21
Variance score random: -0.66
Variance score regress: -0.14

Movie #12, 11    My Favorite Brunette
Name: Name, dtype: object, average rating: 3.42 in 546 reviews
(546, 545)
(546, 1)
X_train : (436, 545) 	
X_test : (110, 545) 	
y_train : (436, 1) 	
y_test : (110, 1) 	

No Clustering
Variance score: -11.52

KMeans Clustering
Variance score mean: -0.05
Variance score mode: -0.26
Variance score random: -0.47
Variance score regress: -1.00

DBSCAN Clustering
Variance score mean: -0.00
Variance score mode: -0.15
Variance score random: -1.02
Variance score regress: -11.52



/Users/Cat/anaconda3/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Spectral Clustering
Variance score mean: -0.43
Variance score mode: -0.60
Variance score random: -1.21
Variance score regress: -1.05

Agglomerative Clustering
Variance score mean: -0.05
Variance score mode: -0.09
Variance score random: -0.54
Variance score regress: -0.16

Movie #13, 12    Lord of the Rings: The Return of the King: Ext...
Name: Name, dtype: object, average rating: 4.55 in 125 reviews
(125, 124)
(125, 1)
X_train : (100, 124) 	
X_test : (25, 124) 	
y_train : (100, 1) 	
y_test : (25, 1) 	

No Clustering
Variance score: -2.51

KMeans Clustering
Variance score mean: 0.03
Variance score mode: -0.66
Variance score random: -1.34
Variance score regress: -0.20

DBSCAN Clustering
Variance score mean: -0.02
Variance score mode: -0.66
Variance score random: -0.46
Variance score regress: -2.51



/Users/Cat/anaconda3/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Spectral Clustering
Variance score mean: -0.02
Variance score mode: -0.66
Variance score random: -1.54
Variance score regress: -0.31

Agglomerative Clustering
Variance score mean: 0.06
Variance score mode: -0.66
Variance score random: -1.34
Variance score regress: -0.11

Movie #14, 13    Nature: Antarctica
Name: Name, dtype: object, average rating: 3.03 in 118 reviews
(118, 117)
(118, 1)
X_train : (94, 117) 	
X_test : (24, 117) 	
y_train : (94, 1) 	
y_test : (24, 1) 	

No Clustering
Variance score: -2073.74

KMeans Clustering
Variance score mean: -0.01
Variance score mode: -0.21
Variance score random: -1.16
Variance score regress: -801.08

DBSCAN Clustering
Variance score mean: -0.00
Variance score mode: -0.00
Variance score random: -0.87
Variance score regress: -2073.74

Spectral Clustering
Variance score mean: -0.09
Variance score mode: -0.00
Variance score random: -0.80
Variance score regress: -13.29

Agglomerative Clustering
Variance score mean: -0.84
Variance score mode: -2.16
Var

/Users/Cat/anaconda3/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [26]:
movie_results_dict[3]

{'data_time': 1.1898272037506104,
 'mse': 1.6863325,
 'name': '2    Character\nName: Name, dtype: object',
 'r2': -0.74125852269030212,
 'regr': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
 'regr_time': 4.725301027297974}

In [22]:
review_nums = []
for i in range(17700):
    num_reviews = ratings_small[:,i].count_nonzero()
    review_nums.append((i, num_reviews, np.sum(ratings_small[:,i]) / num_reviews))

/Users/Cat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [24]:
s = sorted(review_nums, key=lambda x: x[2])
for movie_id, num, avg_review in s[-20:]:
    print ('%s\t%s\t%s' % (num, avg_review, movie_by_id[movie_id]))

681	4.5389133627	Fruits Basket (2001)
17292	4.54256303493	The Simpsons: Season 5 (1993)
92470	4.54370065967	Star Wars: Episode V: The Empire Strikes Back (1980)
134284	4.54512078878	Lord of the Rings: The Return of the King (2003)
125	4.552	Lord of the Rings: The Return of the King: Extended Edition: Bonus Material (2003)
1883	4.55443441317	Inu-Yasha (2000)
8426	4.58129598861	The Simpsons: Season 6 (1994)
6621	4.58238936717	Arrested Development: Season 2 (2004)
220	4.58636363636	Ghost in the Shell: Stand Alone Complex: 2nd Gig (2005)
1238	4.59208400646	Veronica Mars: Season 1 (2004)
139660	4.59338393241	The Shawshank Redemption: Special Edition (1994)
89	4.59550561798	Tenchi Muyo! Ryo Ohki (1995)
25	4.6	Trailer Park Boys: Season 4 (2003)
75	4.6	Trailer Park Boys: Season 3 (2003)
1633	4.60502143295	Fullmetal Alchemist (2004)
1747	4.63880938752	Battlestar Galactica: Season 1 (2004)
7249	4.67098910195	Lost: Season 1 (2004)
74912	4.70261106365	Lord of the Rings: The Two Towers: Extended Ed